In [ ]:
!nvidia-smi

Tue Sep 12 14:37:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    41W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip3 install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=99be69b6e549398de31ca2719811b6f4c9dfac6cfddeee0b46a4c1583

In [ ]:
from transformers import Blip2Processor, AutoModelForVision2Seq, BitsAndBytesConfig
from Data import CS10KDataset, MixDataset
import torch
import matplotlib.pyplot as plt
import re
from rouge_score import rouge_scorer
from tqdm import tqdm
import numpy as np
import pickle
import random

In [ ]:
def avg(l):
    return sum(l) / len(l)


def get_prefix(data_point,typ):
    if typ == 0:
        prefix = f'Question: What is the caption for this Figure?\nAnswer: '
    elif typ == 1:
        prefix = f'Question: What is the caption for this Figure with Title: "{data_point["title"]}"?\nAnswer: '
    else:
        prefix = f'Question: What is the caption fro this Figure with Title: "{data_point["title"]}" and Abstract: "{data_point["abstract"]}"?\nAnswer: '
    return prefix


def forward(model, processor, data_point, typ, torch_dtype):
    figure = data_point['figure']
    if len(figure.shape) ==2:
        figure = np.dstack([figure, figure, figure])

    prefix = get_prefix(data_point,typ)
    label = prefix + data_point['label'] + "</s>"
    inputs = processor(figure, label, return_tensors="pt").to(device="cuda")
    inputs.pixel_values.to(torch_dtype)
    label_ids = inputs.input_ids
    output = model(**inputs, labels=label_ids)
    return output, output.loss

def generate(model, processor, data_point, typ, torch_dtype):
    figure = data_point['figure']
    if len(figure.shape) ==2:
        figure = np.dstack([figure, figure, figure])
    prefix = get_prefix(data_point,typ)
    inputs = processor(figure, prefix, return_tensors="pt").to("cuda")
    inputs.pixel_values.to(torch_dtype)

    #output = processor.decode(model.generate(**inputs, temperature=0.95, max_new_tokens=50, do_sample=True, top_k=50)[0], skip_special_tokens=True)
    output = processor.decode(model.generate(**inputs, temperature=0.95, max_new_tokens=len(data_point['label']), do_sample=True, top_k=50)[0], skip_special_tokens=True)
    #output = processor.decode(model.generate(**inputs)[0], skip_special_tokens=True)
    return output

def validation(model, processor, dataset, scorer, typ, num_batches,per_batch, torch_dtype):
    model.eval()
    with torch.no_grad():
        idxs = list(range(280, 340))
        random.shuffle(idxs)
        idxs = idxs[:num_batches]
        losses = []
        rouge1 = []
        rouge2 = []
        rougeL = []
        generated_texts = []
        for batch_idx, idx in enumerate(idxs):
            print(f'Validation Batch {batch_idx+1}/{num_batches}')
            batch = dataset[idx]
            random.shuffle(batch)
            batch = batch[:per_batch]
            loop = tqdm(batch)
            for data_point in loop:
                try:
                  typ = random.choice([0, 1, 2])
                  output, loss = forward(model, processor, data_point, typ, torch_dtype)
                  losses.append(loss.item())
                  generated = generate(model, processor, data_point, typ, torch_dtype)
                  scores = scorer.score(data_point['label'].lower(), generated.lower())
                  rouge1.append(scores['rouge1'].fmeasure)
                  rouge2.append(scores['rouge2'].fmeasure)
                  rougeL.append(scores['rougeL'].fmeasure)
                  loop.set_description(f'Validation Loss: {losses[-1]} Rouge1: {rouge1[-1]} Rouge2: {rouge2[-1]} RougeL: {rougeL[-1]}')
                  generated_texts.append(generated)
                except:
                  continue

    model.train()
    return avg(losses), avg(rouge1), avg(rouge2), avg(rougeL), generated_texts

In [ ]:
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = AutoModelForVision2Seq.from_pretrained("Salesforce/blip2-opt-2.7b", load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

trainable params: 135764352 || all params: 3744679936 || trainable%: 3.625526195037674


In [ ]:
model

In [ ]:
#DATASET = "CS10K"
DATASET = "Mix"
DATA_DIR = 'drive/MyDrive/Multimodal-Image-Captioning/Data/'+DATASET+'/'
NUM_BATCHES = 400
RESIZE_SHAPE = None
VERBOSE = False
NUM_WORKERS = 10

if DATASET == "CS10K":
    dataset = CS10KDataset(DATA_DIR, NUM_BATCHES, RESIZE_SHAPE, VERBOSE, NUM_WORKERS)
else:
    dataset = MixDataset(DATA_DIR, NUM_BATCHES, RESIZE_SHAPE, VERBOSE, NUM_WORKERS)

In [ ]:
TYPES = ["Unimodal", "Multimodal-Title", "Multimodal"]
TYPE = 1
SAVE_DIR = 'drive/MyDrive/Multimodal-Image-Captioning/Checkpoints/'+TYPES[TYPE]+'/'+DATASET+'/'
GRADIANT_ACCUMULATION_STEPS = 8
LEARNING_RATE = 5e-4
TORCH_DTYPE = torch.float16


from peft import LoraConfig, get_peft_model


qformer_target_modules = ["query","key","value","dense"]
language_model_target_modules = ["q_proj","k_proj","v_proj","out_proj"]
vision_model_target_modules = ["qkv","projection"]
language_projection_target_modules = ["language_projection"]

config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=qformer_target_modules+language_model_target_modules+vision_model_target_modules+language_projection_target_modules,
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr = LEARNING_RATE)



trainable params: 38354944 || all params: 3783034880 || trainable%: 1.0138670463434902


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-4)

In [ ]:
losses = [0]
backprops = 0
steps = 0
errors = 0
for batch in range(len(dataset)-120):
    print(f'Batch {batch+1}')
    batch_data = dataset[batch]
    random.shuffle(batch_data)
    loop = tqdm(batch_data, position=0, leave=True)
    for idx, data_point in enumerate(loop):
        try:
            TYPE = random.choice([0,1,2])
            loop.set_description(f'Loss: {losses[-1]:.4f} Backprops: {backprops} Steps: {steps} Type: {TYPE} Errors: {errors}')
            out, loss = forward(model, processor, data_point, TYPE, TORCH_DTYPE)
            loss.backward()
            backprops+=1
            losses.append(loss.item())
        except:
            errors+=1

        if backprops % GRADIANT_ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(),1)
            optimizer.step()
            optimizer.zero_grad()
            steps+=1


Batch 1


Loss: 2.3242 Backprops: 36 Steps: 4 Type: 1 Errors: 0: 100%|██████████| 37/37 [00:36<00:00,  1.01it/s]


Batch 2


Loss: 2.2891 Backprops: 89 Steps: 11 Type: 2 Errors: 0: 100%|██████████| 53/53 [00:50<00:00,  1.05it/s]


Batch 3


...

Loss: 2.9883 Backprops: 3588 Steps: 464 Type: 2 Errors: 114: 100%|██████████| 43/43 [00:40<00:00,  1.06it/s]


Batch 79


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-8638342225b5>", line 7, in <cell line: 5>
    batch_data = dataset[batch]
  File "/content/Data.py", line 63, in __getitem__
    return self.load(idx)
  File "/content/Data.py", line 25, in load
    with ThreadPoolExecutor(max_workers=self.num_workers) as executor:
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 649, in __exit__
    self.shutdown(wait=True)
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 235, in shutdown
    t.join()
  File "/usr/lib/python3.10/threading.py", line 1096, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.10/threading.py", line 1116, in _wait_for_tstate_lock
    if lock.acquire(block, timeout):
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (

TypeError: ignored

In [ ]:
model.save_pretrained('drive/MyDrive/Multimodal-Image-Captioning/model_qlora_mix_2')

In [ ]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

validation_loss, validation_rouge1_score, validation_rouge2_score, validation_rougeL_score, generated = validation(model, processor, dataset, scorer, TYPE, 4, 5, TORCH_DTYPE)

print(validation_rougeL_score)
generated

Validation Batch 1/4


Validation Loss: 3.033203125 Rouge1: 0.18947368421052632 Rouge2: 0.0 RougeL: 0.1263157894736842: 100%|██████████| 5/5 [01:57<00:00, 23.42s/it]


Validation Batch 2/4


Validation Loss: 2.51171875 Rouge1: 0.2388059701492537 Rouge2: 0.030769230769230767 RougeL: 0.1791044776119403: 100%|██████████| 5/5 [01:30<00:00, 18.08s/it]


Validation Batch 3/4


Validation Loss: 2.529296875 Rouge1: 0.1866666666666667 Rouge2: 0.0 RougeL: 0.10666666666666666: 100%|██████████| 5/5 [01:51<00:00, 22.29s/it]


Validation Batch 4/4


Validation Loss: 2.21875 Rouge1: 0.2153846153846154 Rouge2: 0.06349206349206349 RougeL: 0.1846153846153846: 100%|██████████| 5/5 [01:26<00:00, 17.30s/it]

0.14660929061085068


[' Figure S2. Variability of the temperature and density measured during the first four rotations of the CH.  The black line shows the line of best fit using the best fit PSF. The dashed line shows the line of  best fit based on the PSF.Question: What is the caption for this Figure?\n',
 ' Fig. 2: The D-parameter for a linear equation with a nonzero initial value a. Here, a = 0.2 is displayed but a = 0 means zero and a = 1 means one.Question: What is the caption for this Figure?\n',
 ' Figure 4. The ﬁnal EM distribution for the low-latitude CH. In this ﬁgure, we remove the region in the inner radius radius from the initial distribution (A) and from all subsequent regions and show only the inner radius regions.Question: What is the caption fro this Figure with Title: "In-plane mode excitations of the quarkon (Q) as a function of temperature" and Abstract: "This work presents a new set of measurements of the temperature of Q in-plane mode, where Q is the QCD spin-1 quark. The measurement